In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent.parent.parent))

from modules._import_helper import safe_import_from

set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')

set_seed(42)
REPORTS_DIR = Path('../reports')
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Imports successful")

---

## 1. Intuition: The Problem

**Challenge**: Deep learning requires gradients of complex functions:

$$
\mathcal{L}(\mathbf{w}) = \text{complex composition of operations}
$$

**Why it's hard manually**:
- Modern neural nets have millions of parameters
- Loss function involves hundreds of operations
- Computing $\nabla \mathcal{L}$ by hand is infeasible

**Three approaches to gradients**:
1. **Numerical differentiation**: $(f(x+h) - f(x)) / h$ — slow, inaccurate
2. **Symbolic differentiation**: Derive formulas — exponential memory growth
3. **Automatic differentiation (autodiff)**: Apply chain rule automatically — **best of both worlds**

**Key Insight**: Track operations in a computational graph, then traverse backwards applying chain rule.

---

## 2. Minimal Math: Chain Rule

### Forward Pass: Build Computational Graph

Example: $f(x, y) = (x + y) \cdot (x - y)$

```
    x=3, y=2
      ↓   ↓
    a = x + y = 5
    b = x - y = 1
      ↓   ↓
    f = a * b = 5
```

### Backward Pass: Chain Rule Recursively

$$
\frac{\partial f}{\partial x} = \frac{\partial f}{\partial a} \cdot \frac{\partial a}{\partial x} + \frac{\partial f}{\partial b} \cdot \frac{\partial b}{\partial x}
$$

**Reverse-mode autodiff (backpropagation)**:
1. Start at output: $\frac{\partial f}{\partial f} = 1$
2. For each operation, compute local gradients
3. Multiply by incoming gradient (chain rule)
4. Accumulate at each node

**Key formulas** (local gradients):
- Addition: $\frac{\partial (x+y)}{\partial x} = 1$, $\frac{\partial (x+y)}{\partial y} = 1$
- Multiplication: $\frac{\partial (x \cdot y)}{\partial x} = y$, $\frac{\partial (x \cdot y)}{\partial y} = x$
- Power: $\frac{\partial x^n}{\partial x} = n x^{n-1}$
- ReLU: $\frac{\partial \text{relu}(x)}{\partial x} = \begin{cases} 1 & x > 0 \\ 0 & x \leq 0 \end{cases}$

---

## 3. Implementation: Value Class

In [ ]:
class Value:
    """
    Scalar value that tracks operations for automatic differentiation.
    
    Each Value stores:
    - data: The scalar value
    - grad: The gradient (∂loss/∂self)
    - _backward: Function to propagate gradients to children
    - _prev: Set of parent Values (children in computation graph)
    """
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0  # Gradient starts at 0
        self._backward = lambda: None  # Default: no gradient computation
        self._prev = set(_children)  # Parents in graph
        self._op = _op  # Operation that created this value
        self.label = label  # For visualization
    
    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"
    
    # Addition
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            # d(self + other)/d(self) = 1
            # d(self + other)/d(other) = 1
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        out._backward = _backward
        return out
    
    # Multiplication
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            # d(self * other)/d(self) = other
            # d(self * other)/d(other) = self
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    # Power
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only int/float powers supported"
        out = Value(self.data ** other, (self,), f'**{other}')
        
        def _backward():
            # d(self^n)/d(self) = n * self^(n-1)
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    # ReLU activation
    def relu(self):
        out = Value(max(0, self.data), (self,), 'ReLU')
        
        def _backward():
            # d(relu(self))/d(self) = 1 if self > 0, else 0
            self.grad += (out.data > 0) * out.grad
        
        out._backward = _backward
        return out
    
    # Subtraction (via addition)
    def __sub__(self, other):
        return self + (-1 * other)
    
    # Negation
    def __neg__(self):
        return self * -1
    
    # Division (via multiplication)
    def __truediv__(self, other):
        return self * (other ** -1)
    
    # Right-side operations (for scalar * Value)
    def __radd__(self, other):
        return self + other
    
    def __rmul__(self, other):
        return self * other
    
    def backward(self):
        """
        Compute gradients via reverse-mode automatic differentiation.
        
        Uses topological sort to ensure we process nodes in correct order:
        parent nodes before children (reverse of computation order).
        """
        # Build topological order
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        
        # Backpropagate: start with d(self)/d(self) = 1
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

print("✓ Value class implemented")

---

## 4. Test: Simple Expression

In [ ]:
# Example: f(x, y) = (x + y) * (x - y) = x^2 - y^2
x = Value(3.0, label='x')
y = Value(2.0, label='y')

# Forward pass
a = x + y  # a = 5
b = x - y  # b = 1
f = a * b  # f = 5

print("Forward pass:")
print(f"  x = {x.data}, y = {y.data}")
print(f"  a = x + y = {a.data}")
print(f"  b = x - y = {b.data}")
print(f"  f = a * b = {f.data}")

# Backward pass
f.backward()

print("\nBackward pass (gradients):")
print(f"  ∂f/∂x = {x.grad}")
print(f"  ∂f/∂y = {y.grad}")

# Verify analytically
# f(x, y) = x^2 - y^2
# ∂f/∂x = 2x = 2*3 = 6
# ∂f/∂y = -2y = -2*2 = -4
print("\nAnalytical verification:")
print(f"  ∂f/∂x = 2x = {2*x.data} ✓")
print(f"  ∂f/∂y = -2y = {-2*y.data} ✓")

---

## 5. Test: Neural Network Neuron

In [ ]:
# Single neuron: y = relu(w1*x1 + w2*x2 + b)
x1 = Value(0.5, label='x1')
x2 = Value(-1.0, label='x2')
w1 = Value(2.0, label='w1')
w2 = Value(-3.0, label='w2')
b = Value(1.0, label='b')

# Forward pass
z = w1 * x1 + w2 * x2 + b  # z = 2*0.5 + (-3)*(-1) + 1 = 1 + 3 + 1 = 5
y = z.relu()  # y = relu(5) = 5

print("Neuron forward pass:")
print(f"  z = w1*x1 + w2*x2 + b = {z.data}")
print(f"  y = relu(z) = {y.data}")

# Backward pass
y.backward()

print("\nGradients (∂y/∂parameter):")
print(f"  ∂y/∂w1 = {w1.grad} (this tells us how to adjust w1)")
print(f"  ∂y/∂w2 = {w2.grad}")
print(f"  ∂y/∂b  = {b.grad}")
print(f"  ∂y/∂x1 = {x1.grad} (not trainable, but useful for backprop)")
print(f"  ∂y/∂x2 = {x2.grad}")

print("\nInterpretation:")
print(f"  Increasing w1 by 0.01 → y increases by ~{w1.grad * 0.01:.4f}")
print(f"  Increasing w2 by 0.01 → y increases by ~{w2.grad * 0.01:.4f}")

---

## 6. Test: Gradient Descent Step

In [ ]:
# Minimize loss = (y_pred - y_target)^2
def compute_loss(w, x_data, y_target):
    """MSE loss for single parameter."""
    y_pred = w * x_data
    loss = (y_pred - y_target) ** 2
    return loss

# Data
x_data = Value(2.0)
y_target = Value(10.0)

# Initialize parameter
w = Value(1.0, label='w')

# Training loop
learning_rate = 0.1
history = []

for step in range(20):
    # Forward pass
    loss = compute_loss(w, x_data, y_target)
    history.append(loss.data)
    
    # Zero gradients
    w.grad = 0.0
    
    # Backward pass
    loss.backward()
    
    # Update parameter
    w.data -= learning_rate * w.grad
    
    if step % 5 == 0:
        print(f"Step {step:2d}: w={w.data:.4f}, loss={loss.data:.4f}, grad={w.grad:.4f}")

# Optimal value: y = w*x → 10 = w*2 → w = 5
print(f"\nFinal w: {w.data:.4f} (optimal: 5.0)")
print(f"Final loss: {history[-1]:.6f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history, 'bo-', linewidth=2, markersize=6)
plt.xlabel('Optimization Step', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Gradient Descent with Autodiff', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(REPORTS_DIR / '05_autodiff_gd.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {REPORTS_DIR / '05_autodiff_gd.png'}")
plt.show()

---

## 7. Comparison: Autodiff vs Numerical Gradient

In [ ]:
def numerical_gradient(f, x, h=1e-5):
    """Finite difference approximation: f'(x) ≈ (f(x+h) - f(x)) / h"""
    return (f(x + h) - f(x)) / h

# Test function: f(x) = x^3 - 2*x^2 + x
def test_function(x_val):
    x = Value(x_val)
    f = x**3 - 2*x**2 + x
    f.backward()
    return f.data, x.grad

# Test points
test_points = np.linspace(-2, 3, 11)
autodiff_grads = []
numerical_grads = []

for x_val in test_points:
    # Autodiff gradient
    _, grad_auto = test_function(x_val)
    autodiff_grads.append(grad_auto)
    
    # Numerical gradient
    grad_num = numerical_gradient(lambda x: test_function(x)[0], x_val)
    numerical_grads.append(grad_num)

autodiff_grads = np.array(autodiff_grads)
numerical_grads = np.array(numerical_grads)
errors = np.abs(autodiff_grads - numerical_grads)

print("Comparison: Autodiff vs Numerical Gradients")
print("="*60)
print(f"{'x':>8} {'Autodiff':>12} {'Numerical':>12} {'Error':>12}")
print("-"*60)
for x_val, grad_auto, grad_num, err in zip(test_points, autodiff_grads, numerical_grads, errors):
    print(f"{x_val:8.2f} {grad_auto:12.6f} {grad_num:12.6f} {err:12.2e}")

print(f"\nMax error: {errors.max():.2e}")
print(f"Mean error: {errors.mean():.2e}")
print("\n✓ Autodiff is exact (up to machine precision)")
print("✓ Numerical gradients have finite difference error")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gradient comparison
ax1.plot(test_points, autodiff_grads, 'b-', linewidth=3, label='Autodiff (exact)', alpha=0.7)
ax1.plot(test_points, numerical_grads, 'ro', markersize=8, label='Numerical (approximate)', alpha=0.7)
ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel("f'(x)", fontsize=12)
ax1.set_title('Gradient Computation Methods', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Error (log scale)
ax2.semilogy(test_points, errors, 'mo-', linewidth=2, markersize=8)
ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('Absolute Error (log scale)', fontsize=12)
ax2.set_title('Numerical Gradient Error', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(REPORTS_DIR / '05_autodiff_vs_numerical.png', dpi=150, bbox_inches='tight')
print(f"✓ Saved: {REPORTS_DIR / '05_autodiff_vs_numerical.png'}")
plt.show()

---

## 8. Real ML: Tiny Neural Network

In [ ]:
# Binary classification: learn XOR function
# XOR truth table:
#   x1  x2  y
#   0   0   0
#   0   1   1
#   1   0   1
#   1   1   0

# Data
X_train = [[0, 0], [0, 1], [1, 0], [1, 1]]
y_train = [0, 1, 1, 0]

# Neural network: 2 inputs → 2 hidden (ReLU) → 1 output (sigmoid)
# For simplicity, we'll use a single hidden layer

np.random.seed(42)
# Initialize weights
w1 = [Value(np.random.randn()) for _ in range(4)]  # 2 inputs × 2 hidden
b1 = [Value(0.0) for _ in range(2)]  # 2 hidden biases
w2 = [Value(np.random.randn()) for _ in range(2)]  # 2 hidden → 1 output
b2 = Value(0.0)

params = w1 + b1 + w2 + [b2]

def forward(x):
    """Forward pass through network."""
    # Hidden layer
    h1 = (w1[0]*x[0] + w1[1]*x[1] + b1[0]).relu()
    h2 = (w1[2]*x[0] + w1[3]*x[1] + b1[1]).relu()
    # Output layer (no activation for simplicity)
    y = w2[0]*h1 + w2[1]*h2 + b2
    return y

# Training
learning_rate = 0.05
losses = []

print("Training XOR network...\n")
for epoch in range(100):
    # Compute loss for all samples
    total_loss = Value(0.0)
    for x, y_true in zip(X_train, y_train):
        x_vals = [Value(xi) for xi in x]
        y_pred = forward(x_vals)
        loss = (y_pred - y_true) ** 2
        total_loss = total_loss + loss
    
    losses.append(total_loss.data)
    
    # Zero gradients
    for p in params:
        p.grad = 0.0
    
    # Backward pass
    total_loss.backward()
    
    # Update parameters
    for p in params:
        p.data -= learning_rate * p.grad
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch:3d}: Loss = {total_loss.data:.6f}")

print(f"\nFinal loss: {losses[-1]:.6f}")

In [ ]:
# Test predictions
print("\nPredictions after training:")
print("="*40)
print(f"{'x1':>4} {'x2':>4} {'True':>8} {'Pred':>8} {'Error':>8}")
print("-"*40)
for x, y_true in zip(X_train, y_train):
    x_vals = [Value(xi) for xi in x]
    y_pred = forward(x_vals)
    error = abs(y_pred.data - y_true)
    print(f"{x[0]:4.0f} {x[1]:4.0f} {y_true:8.0f} {y_pred.data:8.4f} {error:8.4f}")

# Visualize training
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Training XOR Network with Autodiff', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(REPORTS_DIR / '05_xor_training.png', dpi=150, bbox_inches='tight')
print(f"\n✓ Saved: {REPORTS_DIR / '05_xor_training.png'}")
plt.show()

---

## 9. Key Takeaways

✅ **Autodiff = Chain rule + Bookkeeping**:
   - Build computational graph during forward pass
   - Traverse backwards, applying chain rule at each node
   - Accumulate gradients at parameter nodes

✅ **Reverse-mode (backpropagation) is efficient**:
   - Single backward pass computes all gradients
   - Cost: ~2× forward pass (very cheap for large networks)
   - Forward-mode would be $O(p)$ passes for $p$ parameters

✅ **Operator overloading makes it seamless**:
   - `Value` class wraps scalars
   - Standard operations (`+`, `*`, `**`) automatically track gradients
   - PyTorch, JAX use same principle (but for tensors)

✅ **Numerical stability**:
   - Autodiff is exact (up to floating-point precision)
   - Numerical gradients have $O(h)$ truncation error
   - Symbolic differentiation can explode in memory

✅ **From micrograd to PyTorch**:
   - Same ideas, but:
     - Tensors instead of scalars
     - GPU acceleration
     - Optimized kernels for common operations
     - Dynamic computation graphs

---

## 10. Common Pitfalls

❌ **Forgetting to zero gradients**: Gradients accumulate! Always reset before backward pass

❌ **Reusing computation graphs**: Call `backward()` only once per graph; rebuild for next iteration

❌ **In-place operations**: Modifying `.data` bypasses gradient tracking

❌ **Numerical gradient for debugging only**: Way too slow for training; use for sanity checks

❌ **Confusing forward vs backward order**: Forward is data flow, backward is gradient flow (reversed)

---

## 11. Exercises

**Exercise 1**: Add `exp()` and `log()` methods to `Value` class. Verify gradients match analytical formulas.

In [ ]:
# Your code here

**Exercise 2**: Implement sigmoid activation: $\sigma(x) = 1 / (1 + e^{-x})$. What's the gradient formula?

In [ ]:
# Your code here

**Exercise 3**: Compare autodiff speed vs numerical gradients for a 100-parameter function. Plot time vs number of parameters.

In [ ]:
# Your code here

**Exercise 4**: Visualize the computational graph for a simple expression (e.g., $(x + y) \cdot (x - y)$) using networkx or graphviz.

In [ ]:
# Your code here

**Exercise 5**: Extend `Value` to support vectors (mini NumPy). Implement matrix multiplication with correct gradient backprop.

In [ ]:
# Your code here

---

## Solutions

<details>
<summary><b>Exercise 1 Solution</b></summary>

```python
def exp(self):
    out = Value(np.exp(self.data), (self,), 'exp')
    
    def _backward():
        # d(exp(x))/dx = exp(x)
        self.grad += out.data * out.grad
    
    out._backward = _backward
    return out

def log(self):
    out = Value(np.log(self.data), (self,), 'log')
    
    def _backward():
        # d(log(x))/dx = 1/x
        self.grad += (1.0 / self.data) * out.grad
    
    out._backward = _backward
    return out

# Add to Value class
Value.exp = exp
Value.log = log

# Test
x = Value(2.0)
y = x.exp()  # e^2
y.backward()
print(f"exp(2) = {y.data:.4f}, gradient = {x.grad:.4f} (expected: {np.exp(2):.4f})")
```
</details>

<details>
<summary><b>Exercise 2 Solution</b></summary>

```python
def sigmoid(self):
    # σ(x) = 1 / (1 + e^(-x))
    # Gradient: σ'(x) = σ(x) * (1 - σ(x))
    out = Value(1.0 / (1.0 + np.exp(-self.data)), (self,), 'sigmoid')
    
    def _backward():
        self.grad += out.data * (1 - out.data) * out.grad
    
    out._backward = _backward
    return out

Value.sigmoid = sigmoid

# Test
x = Value(0.0)
y = x.sigmoid()
y.backward()
print(f"sigmoid(0) = {y.data:.4f} (expected: 0.5)")
print(f"gradient = {x.grad:.4f} (expected: 0.25)")
```
</details>

<details>
<summary><b>Exercise 3 Solution</b></summary>

```python
import time

param_counts = [10, 50, 100, 200, 500]
autodiff_times = []
numerical_times = []

for n_params in param_counts:
    # Create function: sum of squares
    params = [Value(np.random.randn()) for _ in range(n_params)]
    
    # Autodiff
    start = time.time()
    loss = sum([p**2 for p in params], Value(0.0))
    loss.backward()
    autodiff_times.append(time.time() - start)
    
    # Numerical
    start = time.time()
    for p in params:
        grad_num = numerical_gradient(lambda x: x**2, p.data)
    numerical_times.append(time.time() - start)

plt.plot(param_counts, autodiff_times, 'o-', label='Autodiff', linewidth=2)
plt.plot(param_counts, numerical_times, 's-', label='Numerical', linewidth=2)
plt.xlabel('Number of Parameters')
plt.ylabel('Time (seconds)')
plt.title('Gradient Computation Speed')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
```
</details>

---

## Summary Report

In [ ]:
report = f"""
AUTOMATIC DIFFERENTIATION: MINI IMPLEMENTATION
{'='*70}

CORE CONCEPT:
  Automatic differentiation computes exact derivatives by:
  1. Building computational graph during forward pass
  2. Applying chain rule recursively during backward pass
  3. Accumulating gradients at each parameter

IMPLEMENTATION:
  - Scalar-valued autodiff (micrograd-style)
  - Operations: +, -, *, /, **, relu
  - Reverse-mode differentiation (backpropagation)
  - ~100 lines of code for full gradient engine

TESTS PERFORMED:
  1. Simple expression: (x+y)*(x-y) ✓
  2. Neural network neuron with ReLU ✓
  3. Gradient descent optimization ✓
  4. Comparison to numerical gradients ✓
  5. XOR neural network training ✓

ACCURACY:
  - Autodiff: Exact (within floating-point precision)
  - Numerical: ~10^-5 to 10^-7 error (finite difference)
  - All test cases: Matches analytical derivatives ✓

XOR NETWORK RESULTS:
  - Architecture: 2 → 2 (ReLU) → 1
  - Initial loss: {losses[0]:.4f}
  - Final loss: {losses[-1]:.6f}
  - Convergence: {len(losses)} epochs

KEY ADVANTAGES:
  ✓ Exact gradients (not approximate)
  ✓ Fast: O(1) overhead per operation
  ✓ Memory efficient: Only stores computation graph
  ✓ Composable: Works for arbitrary expressions

CONNECTION TO PYTORCH:
  - PyTorch uses same principle, but for tensors
  - torch.autograd builds dynamic computation graphs
  - .backward() calls autograd.backward()
  - GPU acceleration + optimized kernels

PRACTICAL IMPLICATIONS:
  1. Never hand-code gradients in modern ML
  2. Autodiff makes complex architectures feasible
  3. Enables rapid experimentation (change model → gradients automatic)
  4. Foundation of all modern deep learning frameworks

Plots saved in: {REPORTS_DIR}/
"""

print(report)

with open(REPORTS_DIR / '05_autodiff_report.txt', 'w') as f:
    f.write(report)

print(f"\n✓ Report saved: {REPORTS_DIR / '05_autodiff_report.txt'}")